In [1]:
import numpy as np
from scipy.spatial import KDTree



In [2]:
def chamfer_distance(A, B):
    """
    Computes the chamfer distance between two sets of points A and B.
    """
    tree = KDTree(B)
    dist_A = tree.query(A)[0]
    tree = KDTree(A)
    dist_B = tree.query(B)[0]
    return np.mean(dist_A) + np.mean(dist_B)

In [3]:
import numpy as np

def chamfer_distance(points1, points2):
    """
    Compute Chamfer Distance between two 3D point clouds.

    Args:
        points1: numpy array of shape (N, 3)
        points2: numpy array of shape (M, 3)

    Returns:
        chamfer_dist: scalar (float)
    """
    # Compute pairwise distances (squared Euclidean)
    diff = np.expand_dims(points1, axis=1) - np.expand_dims(points2, axis=0)
    dist_sq = np.sum(diff ** 2, axis=2)
    
    # For each point in points1, find the nearest neighbor in points2
    min_dist_1 = np.min(dist_sq, axis=1)
    # For each point in points2, find the nearest neighbor in points1
    min_dist_2 = np.min(dist_sq, axis=0)
    
    # Chamfer Distance (symmetric)
    chamfer_dist = np.mean(min_dist_1) + np.mean(min_dist_2)
    
    return chamfer_dist

# Example usage:
A = np.random.rand(100, 3)
B = np.random.rand(120, 3)

cd = chamfer_distance(A, B)
print("Chamfer Distance:", cd)


Chamfer Distance: 0.03522398037609596


In [10]:
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d  # For reading .ply files

def load_ply_points(file_path):
    """
    Load 3D points from a .ply file using Open3D.
    Returns:
        numpy array of shape (N, 3)
    """
    pcd = o3d.io.read_point_cloud(file_path)
    return np.asarray(pcd.points)

def chamfer_distance_kdtree(points1, points2):
    """
    Compute Chamfer Distance between two point clouds using KDTree.

    Args:
        points1: numpy array of shape (N, 3)
        points2: numpy array of shape (M, 3)

    Returns:
        chamfer_dist: scalar float
    """
    tree1 = cKDTree(points1)
    tree2 = cKDTree(points2)

    dist1, _ = tree1.query(points2)
    dist2, _ = tree2.query(points1)

    chamfer_dist = np.mean(dist1 ** 2) + np.mean(dist2 ** 2)
    return chamfer_dist

# === Example usage ===
file1 = r"scaled_output.ply"
file2 = r"scaled_cube_500.ply"

points1 = load_ply_points(file1)
points2 = load_ply_points(file2)

cd = chamfer_distance_kdtree(points1, points2)
print("Chamfer Distance:", cd)


Chamfer Distance: 88.54750058259765


In [9]:
import numpy as np
import open3d as o3d

pcd_cube = o3d.io.read_point_cloud(r"cube_500.ply")
points_cube = np.asarray(pcd_cube.points)
min_bound_cube = points_cube.min(axis=0)
max_bound_cube = points_cube.max(axis=0)
extent_cube = max_bound_cube - min_bound_cube

pcd_cup = o3d.io.read_point_cloud(r"scaled_output.ply")
points_cup = np.asarray(pcd_cup.points)
min_bound_cup = points_cup.min(axis=0)
max_bound_cup = points_cup.max(axis=0)
extent_cup = max_bound_cup - min_bound_cup

# Step 2
# min_vals = points.min(axis=0)
# max_vals = points.max(axis=0)
# current_size = max_vals - min_vals

# Step 3 - Target sizes
desired_size = extent_cup  # Example target
scale_factors = desired_size / extent_cube

# Step 4 - Scale about center
center = (min_bound_cube + max_bound_cube) / 2
scaled_points = (points_cube - center) * scale_factors + center

# Save
pcd_cube.points = o3d.utility.Vector3dVector(scaled_points)
o3d.io.write_point_cloud("scaled_cube_500.ply", pcd_cube)


True